In [31]:
import numpy as np
import math

# Probabilistic Neural Network with 4 layers
class PNN(object):
    def __init__(self):
        self.L2 = []    # Layer 2 that holds the patterns
        print('Empty PNN created.')

    def train(self, X, y, p=2):
        self.n_ = X.shape[1]  # num of features
        self.p_ = p           # num of classes
        
        # Layer 2 (Pattern): Set up empty lists for each class
        for k in range(self.p_):
            self.L2.append([])    # Using Python's basic lists because ndarray cannot append empty arrays
                                  # Also perhaps we might have to input different data types

        # Enter patterns into Layer 2
        for i in range(X.shape[0]):
            self.L2[y[i]].append(X[i])

        print('PNN with %d classes trained.' % self.p_)

    def crossValidate(self, X, y, sigma=0.5):
        result = self.predict(X, sigma)
        num_correct = sum(result[:, 0] == y)

        print('Cross validation accuracy with sigma %.2f: %.1f%%' % (sigma, num_correct/len(y) * 100))

    def predict(self, X, sigma=0.5):
        m = X.shape[0]
        accL3 = np.zeros((m, self.p_))
        accL4 = np.zeros(m)

        self.sigma_ = sigma    # smoothing parameter, not standard deviation
        self.C1_ = 2 * self.sigma_**2
        C2_ = (math.sqrt(2*math.pi) * self.sigma_) ** (- self.n_)
        
        # Layer 1 (Input): x
        for i in range(m):
            x = X[i]

            # Layer 3 (Averaging): for each class
            self.L3_ = np.zeros(self.p_)
            for k in range(self.p_):
                for ki in range(len(self.L2[k])):
                    self.L3_[k] += self._activation(x, self.L2[k][ki])
                self.L3_[k] /= len(self.L2[k])
                
                # Multiply constant
                self.L3_[k] *= C2_
                accL3[i][k] = self.L3_[k]

            # Layer 4 (Output/Decision): Maxing
            self.L4_ = self.L3_.argmax()
            accL4[i] = self.L4_

        return np.column_stack((accL4, accL3))

    def _activation(self, x, w):
        diff = x - w
        return math.exp( - np.dot(diff, diff) / self.C1_ )


# Normalize to unit length: [0, 1]
# X must be ndarray
def Normalize(X):
    x_max = X.max(axis=0)
    x_min = X.min(axis=0)
    return (X - x_min) / (x_max - x_min)

In [32]:
# If no internet connection:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [3]:
# Elif internet connection:
import pandas as pd

# Let's change this to an already available library or a *.csv file

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
df.tail()

X = df.iloc[:, :4].values

y = df.iloc[:, 4].values
y[:50] = 0
y[50:100] = 1
y[100:] = 2

In [33]:
X_N = Normalize(X)

# Training data
X_tr = np.row_stack((X_N[0:40], X_N[50:90], X_N[100:140]))
y_tr = np.append(np.append(y[0:40], y[50:90]), y[100:140])

# Cross validation data (may be same as training data for a PNN)
X_cv = X_tr
y_cv = y_tr

# Test data
X_tt = np.row_stack((X_N[40:50], X_N[90:100], X_N[140:150]))
y_tt = np.append(np.append(y[40:50], y[90:100]), y[140:150])

In [34]:
pnn = PNN()

Empty PNN created.


In [35]:
pnn.train(X_tr, y_tr, p=3)

PNN with 3 classes trained.


In [36]:
begin = 0.01
end   = 0.20
step  = 0.01

s = begin
while s <= end:
    pnn.crossValidate(X_cv, y_cv, sigma=s)
    s += step

AttributeError: 'PNN' object has no attribute 'C1_'

In [30]:
result = pnn.predict(X_tt, sigma=0.04)
print(result)

[[  0.00000000e+000   5.99323316e+002   6.29656956e-053   4.18308273e-108]
 [  0.00000000e+000   3.40532007e-007   2.19610585e-027   4.67952523e-087]
 [  0.00000000e+000   1.26274829e+002   1.04912290e-046   1.95818887e-105]
 [  0.00000000e+000   3.20079156e+001   1.00349560e-037   2.67781114e-083]
 [  0.00000000e+000   8.14249043e+001   2.17549087e-050   7.26275555e-101]
 [  0.00000000e+000   3.32151701e+002   1.53563278e-032   5.61338047e-088]
 [  0.00000000e+000   1.83844328e+002   4.19223367e-064   3.91916205e-122]
 [  0.00000000e+000   5.59971358e+002   1.64768783e-043   2.65224567e-101]
 [  0.00000000e+000   3.05549132e+002   3.87679476e-059   1.17898134e-117]
 [  0.00000000e+000   5.69547110e+002   1.39723782e-046   5.33427774e-104]
 [  1.00000000e+000   1.60275796e-051   3.62980978e+001   7.12000824e-008]
 [  1.00000000e+000   8.44704358e-063   2.15832359e+002   7.07707084e-001]
 [  1.00000000e+000   3.52126513e-047   1.92454700e+002   2.55407743e-008]
 [  1.00000000e+000   3.2

In [21]:
from numpy.random import seed
random_state = None
seed(random_state)
r = np.random.permutation(len(y))
X_N2, y2 = X_N[r], y[r]

# Training data
X_tr2 = X_N2[:120]
y_tr2 = y2[:120]

# Cross validation data (may be same as training data for a PNN)
X_cv2 = X_tr2
y_cv2 = y_tr2

# Test data
X_tt2 = X_tt
y_tt2 = y_tt

pnn2 = PNN()
pnn2.train(X_tr2, y_tr2, p=3)

Empty PNN created.
PNN with 3 classes trained.


In [22]:
begin = 0.01
end   = 0.20
step  = 0.01

s = begin
while s <= end:
    pnn2.crossValidate(X_cv2, y_cv2, sigma=s)
    s += step

Cross validation accuracy with sigma 0.01: 100.0%
Cross validation accuracy with sigma 0.02: 100.0%
Cross validation accuracy with sigma 0.03: 100.0%
Cross validation accuracy with sigma 0.04: 100.0%
Cross validation accuracy with sigma 0.05: 100.0%
Cross validation accuracy with sigma 0.06: 100.0%
Cross validation accuracy with sigma 0.07: 99.2%
Cross validation accuracy with sigma 0.08: 99.2%
Cross validation accuracy with sigma 0.09: 98.3%
Cross validation accuracy with sigma 0.10: 97.5%
Cross validation accuracy with sigma 0.11: 98.3%
Cross validation accuracy with sigma 0.12: 98.3%
Cross validation accuracy with sigma 0.13: 98.3%
Cross validation accuracy with sigma 0.14: 96.7%
Cross validation accuracy with sigma 0.15: 96.7%
Cross validation accuracy with sigma 0.16: 96.7%
Cross validation accuracy with sigma 0.17: 96.7%
Cross validation accuracy with sigma 0.18: 96.7%
Cross validation accuracy with sigma 0.19: 96.7%


In [23]:
result2 = pnn2.predict(X_tt2, sigma=0.04)
print(result2)

[[  0.00000000e+000   7.18639821e+002   2.33743449e-048   4.78066597e-108]
 [  0.00000000e+000   2.24878338e+002   2.16358177e-027   5.34802883e-087]
 [  0.00000000e+000   4.18088346e+002   2.35787054e-045   2.23793013e-105]
 [  0.00000000e+000   2.54778501e+002   1.52681535e-035   3.06035559e-083]
 [  0.00000000e+000   3.11794983e+002   9.02055402e-054   8.30032874e-101]
 [  0.00000000e+000   5.33164832e+002   3.09858556e-030   6.41529196e-088]
 [  0.00000000e+000   4.49309517e+002   1.02374859e-064   4.47904624e-122]
 [  0.00000000e+000   8.13429306e+002   1.43881799e-041   3.03113790e-101]
 [  0.00000000e+000   5.49609048e+002   3.47883079e-059   1.34740744e-117]
 [  0.00000000e+000   6.03929977e+002   1.35566017e-042   6.09631741e-104]
 [  1.00000000e+000   1.45705273e-051   2.84850934e+002   6.80464065e-008]
 [  1.00000000e+000   7.67954886e-063   4.22336077e+002   8.10680379e-001]
 [  1.00000000e+000   3.20115016e-047   4.31727773e+002   2.88035524e-008]
 [  1.00000000e+000   3.9